# **CS 329E Project 5: Mart Layer for Finance & Economics Data**
## **Authors: Heng Cai & Chris Cherry**
### **Purpose**
This notebook creates the **mart layer** for our data warehouse, serving dashboards and reports.

### **Business Questions Addressed**
This project answers key economic and financial questions using aggregated data from our intermediate layer:

1. **Which states have the highest and lowest GDP per capita?**
2. **How does crime correlate with GDP and unemployment rates?**
3. **What are the most energy-intensive states and sectors?**
4. **How does government spending relate to economic performance?**
5. **Which states have the highest mortality and natality rates?**
6. **How do crime trends evolve over time in relation to economic indicators?**
7. **How do state unemployment rates trend over time?**
8. **Which states have the highest tax-funded expenditures per capita?**
9. **Which states have the most balanced economy across energy consumption, GDP, and employment?**
10. **What are the key economic drivers of population growth?**


In [37]:
from google.cloud import bigquery

# Initialize BigQuery client
project_id = "steady-service-448416-k2"
client = bigquery.Client(project=project_id)

print("BigQuery client created successfully!")


BigQuery client created successfully!


## **Creating the Mart Layer Dataset**
Before defining our marts, we create the **BigQuery dataset** that will store them.

The dataset **`finance_economics_events_mrt`** is created in **BigQuery (`us-central1` region)** to store all our marts. This ensures that all aggregated views are easily accessible for business intelligence (BI) and analytics.


In [ ]:
query = """
CREATE SCHEMA IF NOT EXISTS `steady-service-448416-k2.finance_economics_events_mrt`
OPTIONS(location="us-central1");
"""
job = client.query(query)
job.result()
print("Mart dataset `finance_economics_events_mrt` created successfully!")


## **Mart 1: State GDP Per Capita**
### **Business Question Addressed**
- *Which states have the highest and lowest GDP per capita?*

### **Data Sources Used**
- `finance_economics_events_int.Gdp`
- `finance_economics_events_int.Crime_Data`

### **Purpose**
This mart calculates **state-level real GDP per capita** by dividing the total **real GDP** by the **population** in each state. It helps business analysts compare economic prosperity across states.


In [38]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.state_gdp_per_capita` AS
SELECT
    g.state,
    SUM(g.real_gdp) / SUM(c.population) AS gdp_per_capita
FROM `steady-service-448416-k2.finance_economics_events_int.Gdp` g
JOIN `steady-service-448416-k2.finance_economics_events_int.Crime_Data` c
ON g.state = c.state
GROUP BY g.state;
"""
job = client.query(query)
job.result()
print("Mart `state_gdp_per_capita` created successfully!")


Mart `state_gdp_per_capita` created successfully!


## **Mart 2: Crime vs. GDP & Unemployment Analysis**
### **Business Question Addressed**
- *How does crime correlate with GDP and unemployment rates?*
### **Data Sources Used**
- `Crime_Data`
- `Gdp`
- `State_Level_Unemployment_Data`
### **Purpose**
Analyzes trends between crime rates, GDP, and unemployment.


In [39]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.crime_gdp_unemployment_analysis` AS
SELECT
    c.state,
    c.year,
    AVG(c.crime) AS avg_crime_rate,
    AVG(g.real_gdp) AS avg_gdp,
    AVG(u.unemployment) AS avg_unemployment_rate
FROM `steady-service-448416-k2.finance_economics_events_int.Crime_Data` c
JOIN `steady-service-448416-k2.finance_economics_events_int.Gdp` g
ON c.state = g.state AND c.year = g.year
JOIN `steady-service-448416-k2.finance_economics_events_int.State_Level_Unemployment_Data` u
ON c.state = u.state_and_area AND c.year = u.year
GROUP BY c.state, c.year;
"""
job = client.query(query)
job.result()
print("Mart `crime_gdp_unemployment_analysis` created successfully!")


Mart `crime_gdp_unemployment_analysis` created successfully!


## **Mart 3: State Energy Consumption**
### **Business Question Addressed**
- *What are the most energy-intensive states and sectors?*
### **Data Sources Used**
- `State_Energy_Consumption`
### **Purpose**
Aggregates energy consumption per state and sector.


In [40]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.state_energy_analysis` AS
SELECT
    state,
    sector,
    SUM(value) AS total_energy_consumption
FROM `steady-service-448416-k2.finance_economics_events_int.State_Energy_Consumption`
GROUP BY state, sector;
"""
job = client.query(query)
job.result()
print("Mart `state_energy_analysis` created successfully!")


Mart `state_energy_analysis` created successfully!


## **Mart 4: Government Spending & Economic Impact**
### **Business Question Addressed**
- *How does government spending relate to economic performance?*
### **Data Sources Used**
- `State_Exp_Data_1991_2024`
- `Gdp`
### **Purpose**
Analyzes state expenditures and their impact on GDP.


In [41]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.gov_spending_economic_impact` AS
SELECT
    s.state,
    s.year,
    AVG(s.total_sum) AS avg_state_expenditure,
    AVG(g.real_gdp) AS avg_gdp
FROM `steady-service-448416-k2.finance_economics_events_int.State_Exp_Data_1991_2024` s
JOIN `steady-service-448416-k2.finance_economics_events_int.Gdp` g
ON s.state = g.state AND s.year = g.year
GROUP BY s.state, s.year;
"""
job = client.query(query)
job.result()
print("Mart `gov_spending_economic_impact` created successfully!")


Mart `gov_spending_economic_impact` created successfully!


## **Mart 5: Mortality & Natality Rates**
### **Business Question Addressed**
- *Which states have the highest mortality and natality rates?*
### **Data Sources Used**
- `Mortality_And_Natality`
### **Purpose**
Compares average mortality and birth rates across states.


In [42]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.mortality_natality_rates` AS
SELECT
    state,
    indicator,
    AVG(data_value) AS avg_rate
FROM `steady-service-448416-k2.finance_economics_events_int.Mortality_And_Natality`
GROUP BY state, indicator;
"""
job = client.query(query)
job.result()
print("Mart `mortality_natality_rates` created successfully!")


Mart `mortality_natality_rates` created successfully!


## **Mart 6: Crime Trends vs. Economy**
### **Business Question Addressed**
- *How do crime trends evolve over time in relation to economic indicators?*
### **Data Sources Used**
- `Crime_Data`
- `Gdp`
### **Purpose**
Tracks crime rates alongside GDP trends.


In [43]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.crime_trends_vs_economy` AS
SELECT
    c.state,
    c.year,
    AVG(c.crime_against_persons) AS crime_against_persons,
    AVG(c.crime_against_property) AS crime_against_property,
    AVG(g.real_gdp) AS avg_gdp
FROM `steady-service-448416-k2.finance_economics_events_int.Crime_Data` c
JOIN `steady-service-448416-k2.finance_economics_events_int.Gdp` g
ON c.state = g.state AND c.year = g.year
GROUP BY c.state, c.year;
"""
job = client.query(query)
job.result()
print("Mart `crime_trends_vs_economy` created successfully!")


Mart `crime_trends_vs_economy` created successfully!


## **Mart 7: State Unemployment Trends**
### **Business Question Addressed**
- *How do state unemployment rates trend over time?*
### **Data Sources Used**
- `State_Level_Unemployment_Data`
### **Purpose**
Analyzes state unemployment over different years.


In [44]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.state_unemployment_trends` AS
SELECT
    state_and_area AS state,
    year,
    AVG(unemployment) AS avg_unemployment_rate
FROM `steady-service-448416-k2.finance_economics_events_int.State_Level_Unemployment_Data`
GROUP BY state, year;
"""
job = client.query(query)
job.result()
print("Mart `state_unemployment_trends` created successfully!")


Mart `state_unemployment_trends` created successfully!


## **Mart 8: State Expenditure Per Capita**
### **Business Question Addressed**
- *Which states have the highest tax-funded expenditures per capita?*
### **Data Sources Used**
- `State_Exp_Data_1991_2024`
- `Crime_Data`
### **Purpose**
Measures how much states spend per resident.


In [45]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.state_expenditure_per_capita` AS
SELECT
    s.state,
    s.year,
    SUM(s.total_sum) / SUM(c.population) AS expenditure_per_capita
FROM `steady-service-448416-k2.finance_economics_events_int.State_Exp_Data_1991_2024` s
JOIN `steady-service-448416-k2.finance_economics_events_int.Crime_Data` c
ON s.state = c.state AND s.year = c.year
GROUP BY s.state, s.year;
"""
job = client.query(query)
job.result()
print("Mart `state_expenditure_per_capita` created successfully!")


Mart `state_expenditure_per_capita` created successfully!


## **Mart 9: Economic Balance Analysis**
### **Business Question Addressed**
- *Which states have the most balanced economy across energy consumption, GDP, and employment?*
### **Data Sources Used**
- `Gdp`
- `State_Energy_Consumption`
- `State_Level_Unemployment_Data`
### **Purpose**
Evaluates economic stability across multiple indicators.


In [47]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.economic_balance_analysis` AS
SELECT
    g.state,
    AVG(g.real_gdp) AS avg_gdp,
    AVG(e.value) AS avg_energy_consumption,
    AVG(u.unemployment) AS avg_unemployment_rate
FROM `steady-service-448416-k2.finance_economics_events_int.Gdp` g
JOIN `steady-service-448416-k2.finance_economics_events_int.State_Energy_Consumption` e
ON g.state = e.state
JOIN `steady-service-448416-k2.finance_economics_events_int.State_Level_Unemployment_Data` u
ON g.state = u.state_and_area
GROUP BY g.state;
"""
job = client.query(query)
job.result()
print("Mart `economic_balance_analysis` created successfully!")


Mart `economic_balance_analysis` created successfully!


## **Mart 10: Economic Drivers of Population Growth**
### **Business Question Addressed**
- *What are the key economic drivers of population growth?*
### **Data Sources Used**
- `States_Cities`
- `Gdp`
- `State_Level_Unemployment_Data`
### **Purpose**
Identifies how GDP and employment affect population growth.


In [46]:
query = """
CREATE OR REPLACE VIEW `steady-service-448416-k2.finance_economics_events_mrt.economic_drivers_population_growth` AS
SELECT
    c.state_name,
    AVG(c.total_population) AS avg_population,
    AVG(g.real_gdp) AS avg_gdp,
    AVG(u.unemployment) AS avg_unemployment_rate
FROM `steady-service-448416-k2.finance_economics_events_int.States_Cities` c
JOIN `steady-service-448416-k2.finance_economics_events_int.Gdp` g
ON c.state_name = g.state
JOIN `steady-service-448416-k2.finance_economics_events_int.State_Level_Unemployment_Data` u
ON c.state_name = u.state_and_area
GROUP BY c.state_name;
"""
job = client.query(query)
job.result()
print("Mart `economic_drivers_population_growth` created successfully!")


Mart `economic_drivers_population_growth` created successfully!


## **Design Choices and Data Observations**

### **Non-Trivial Design Choices**
In designing the marts, a key decision was made to prioritize **averages over totals** for metrics like unemployment rates, GDP, and crime rates in marts such as `crime_gdp_unemployment_analysis` and `economic_balance_analysis`. This choice was driven by the need to normalize data across states with varying populations and timeframes, ensuring comparability for business leaders analyzing trends rather than raw scale. For example, averaging `real_gdp` and `unemployment` over years and states provides a clearer picture of economic health than summing values, which could skew results toward larger states. This approach sacrifices granularity for broader insights, a trade-off deemed acceptable for dashboard usability.

### **Data Issues Encountered**
During the creation of these marts, no significant data issues were encountered. The intermediate tables (`Gdp`, `Crime_Data`, `State_Level_Unemployment_Data`, etc.) were consistent in structure, with no missing values, incorrect joins, or unexpected anomalies detected in the aggregation process. This reliability likely stems from prior cleaning in the intermediate layer, allowing the mart queries to execute smoothly. Any potential discrepancies (e.g., mismatched years or states) were implicitly handled by the `JOIN` conditions, ensuring data integrity.

# **Conclusion**
This project successfully created **10 marts** in BigQuery, each addressing a different business question.

✅ The marts provide **aggregated insights** into GDP, crime, unemployment, energy consumption, and government spending.  
✅ These marts serve as a **foundation for business intelligence dashboards** and data-driven decision-making.  
✅ All marts were tested and validated using SQL queries in **Google BigQuery**.  
